Оценка RAG

In [2]:
from typing import List
import os
import json
import numpy as np
from react_agent.tools import search  # ваш инструмент с Qdrant + fastembed
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy



def retrieve(query: str, top_k: int = 10) -> List[str]:
    """
    Вызов Qdrant-инструмента search() для получения до top_k фрагментов контекста.
    """
    raw = search(query)
    # search() возвращает строки, разделённые сериями ===== и заголовками источников
    # Разбиваем по заголовку каждого результата
    parts = [part.strip() for part in raw.split("==========Актуально в период:") if part.strip()]
    # Берём первые top_k результатов
    return parts[:top_k]


Создание датасета

In [ ]:
dataset = []
questions = open('../data/testing_data/test_cases.txt').read().splitlines()
for q in questions:
    contexts = retrieve(q)
    dataset.append({
        "user_input": q,
        "retrieved_contexts": contexts,
            #"response": response
    })

In [ ]:
from ragas import EvaluationDataset
eval_ds = EvaluationDataset.from_list(dataset)


Оценка

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy, AnswerCorrectness, AnswerCompleteness, ContextPrecision, ContextRecall
llm = ChatOpenAI(
        model="gpt://" + os.environ["FOLDER_ID"] + '/yandexgpt/rc',
        api_key=os.environ["API_KEY"],
        temperature=0.1,
        base_url="https://llm.api.cloud.yandex.net/v1"
    )
evaluator = LangchainLLMWrapper(llm)
metrics = [
    LLMContextRecall(),
    Faithfulness(),
    ResponseRelevancy(),
    AnswerCorrectness(),
    AnswerCompleteness(),
    ContextPrecision(),
    ContextRecall()
]
results = evaluate(
    dataset=eval_ds,
    metrics=metrics,
    llm=evaluator
)
print(results)
